# Classiq Coding Competition Spring 2022
### First Place MCX Submission by Soshun Naito

### DESCRIPTION(S)

I constructed 14-input MCX using some CCCX and CCCCX gates. The depth of CCCX and CCCCX gates can be reduced because most of them don't require keeping the control qubits in the same state.

I used various types of CCX/CCCX/CCCCX gates and connected them to construct a 14-controlled X gate. When minimizing the circuit depth, I tested all possibilities and found my solution.

In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import *
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, QuantumRegister
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
# Reduced Toffoli gate.
# This gate doesn't preserve the states of c0 and c1, but can remove unnecessary CNOT gates.
def Toffoli_left(qc, c0, c1, t):
    qc.h(t)
    qc.t(t)
    qc.cx(c1, t)
    qc.tdg(t)
    qc.cx(c0, t)
    qc.t(t)
    qc.cx(c1, t)
    qc.tdg(t)
    qc.cx(c0, t)
    qc.h(t)
    
# After Applying Toffoli_left gate, this gate can be used for uncomputation.
def Toffoli_right(qc, c0, c1, t):
    qc.h(t)
    qc.cx(c0, t)
    qc.t(t)
    qc.cx(c1, t)
    qc.tdg(t)
    qc.cx(c0, t)
    qc.t(t)
    qc.cx(c1, t)
    qc.tdg(t)
    qc.h(t)
    
def CCX_full(qc, c0, c1, t):
    qc.h(t)
    qc.cx(c1, t)
    qc.tdg(t)
    qc.cx(c0, c1)
    qc.cx(c0, t)
    qc.tdg(c1)
    qc.cx(c0, c1)
    qc.t(t)
    qc.t(c0)
    qc.cx(c1, t)
    qc.t(c1)
    qc.tdg(t)
    qc.cx(c0, t)
    qc.t(t)
    qc.h(t)

In [ ]:
def CCCX_full(qc, c0, c1, c2, t):
    qc.h(t)
    qc.rz(pi / 8, t)
    
    qc.cx(c2, c1)
    qc.rz(-pi / 8, c1)
    qc.cx(c1, t)
    qc.cx(c2, t)
    qc.cx(c0, c1)
    qc.rz(pi / 8, c1)
    qc.cx(c0, c1)
    
    qc.cx(c2, c1)
    qc.rz(-pi / 8, t)
    qc.cx(c0, t)
    qc.rz(pi / 8, t)
    qc.cx(c1, t)
    qc.rz(-pi / 8, t)
    qc.cx(c2, c0)
    qc.rz(-pi / 8, c0)
    qc.cx(c2, c0)
    qc.cx(c2, t)    
    qc.rz(pi / 8, c2)
    qc.rz(pi / 8, t)
    qc.cx(c0, c1)
    qc.rz(-pi / 8, c1)
    qc.rz(pi / 8, c0)
    qc.cx(c0, c1)
    qc.cx(c1, t)
    qc.rz(-pi / 8, t)
    qc.rz(pi / 8, c1)
    
    qc.cx(c0, t)
    qc.rz(pi / 8, t)
    qc.cx(c1, t)
    qc.rz(-pi / 8, t)
    qc.cx(c2, t)
    qc.h(t)

In [ ]:
pi = np.pi
def CCX(qc, c0, c1, t):
    a = pi / 4
    qc.ry(a, t)
    qc.cx(c1, t)
    qc.ry(a, t)
    qc.cx(c0, t)
    qc.ry(-a, t)
    qc.cx(c1, t)
    qc.ry(-a, t)
    
def CCCX(qc, c0, c1, c2, t, mode="full"):
    a = pi / 4
    qc.ry(a, t)
    qc.cx(c2, t)
    qc.ry(a, t)
    if(mode=="left"): Toffoli_left(qc, c0, c1, t)
    if(mode=="right"): Toffoli_right(qc, c0, c1, t)
    if(mode=="full"): qc.ccx(c0, c1, t)
    qc.ry(-a, t)
    qc.cx(c2, t)
    qc.ry(-a, t)
    
def CCCX2(qc, c0, c1, c2, t):
    a = pi / 8
    qc.ry(a * 3, t)
    qc.cx(c2, t)
    qc.ry(a, t)
    qc.cx(c1, t)
    qc.ry(-a, t)
    qc.cx(c2, t)
    qc.ry(a, t)
    qc.cx(c0, t)
    qc.ry(-a, t)
    qc.cx(c2, t)
    qc.ry(a, t)
    qc.cx(c1, t)
    qc.ry(-a, t)
    qc.cx(c2, t)
    qc.ry(-a * 3, t)
    
def CCCCX(qc, c0, c1, c2, c3, t, mode = "full"):
    a = pi / 8
    qc.ry(a * 3, t)
    qc.cx(c3, t)
    qc.ry(a, t)
    qc.cx(c2, t)
    qc.ry(-a, t)
    qc.cx(c3, t)
    qc.ry(a, t)
    if(mode=="left"): Toffoli_left(qc, c0, c1, t)
    if(mode=="right"): Toffoli_right(qc, c0, c1, t)
    if(mode=="full"): qc.ccx(c0, c1, t)
    qc.ry(-a, t)
    qc.cx(c3, t)
    qc.ry(a, t)
    qc.cx(c2, t)
    qc.ry(-a, t)
    qc.cx(c3, t)
    qc.ry(-a * 3, t)
    
def CCCCX2(qc, c0, c1, c2, c3, t, a0):
    a = pi / 8
    qc.ry(a * 3, t)
    qc.cx(c3, t)
    qc.ry(a, t)
    qc.cx(c2, t)
    qc.ry(-a, t)
    qc.cx(c3, t)
    qc.ry(a, t)
    CCX(qc, c0, c1, a0)
    qc.cx(a0, t)
    CCX(qc, c0, c1, a0)
    qc.ry(-a, t)
    qc.cx(c3, t)
    qc.ry(a, t)
    qc.cx(c2, t)
    qc.ry(-a, t)
    qc.cx(c3, t)
    qc.ry(-a * 3, t)

In [ ]:
X = QuantumRegister(size=14,name="x")
Y = QuantumRegister(size=1,name="y")
A = QuantumRegister(size=5, name="a")
XC = ClassicalRegister(size=14, name="xc")
YC = ClassicalRegister(size=1, name="yc")
AC = ClassicalRegister(size=5, name="ac")

ondebug = True
ondebug = False # Enable this line when submitting

if(ondebug): QC = QuantumCircuit(X,Y,A,XC,YC,AC)
else: QC = QuantumCircuit(X,Y,A)

approach = 2

# initial state preparation (for checking logical consistancy)
if(ondebug):
    QC.h(X)
    QC.barrier()

# state concentration
if(approach == 1):
    CCCCX2(QC, X[0], X[1], X[2], X[3], A[0], A[4])
    CCCCX(QC, X[4], X[5], X[6], X[7], A[1], "left")
    CCX(QC, A[1], A[0], A[4])
    CCCX(QC, X[8], X[9], X[10], A[2], "left")
    CCCCX(QC, A[2], X[11], X[12], X[13], A[3], "left")
if(approach == 2):
    CCCCX(QC, X[0], X[1], X[2], X[3], A[0], "left")
    CCCCX2(QC, X[4], X[5], X[6], X[7], A[1], A[4])
    CCCX(QC, X[8], X[9], X[10], A[2], "left")
    CCCCX(QC, A[2], X[11], X[12], X[13], A[3], "left")
    CCX(QC, A[0], A[1], A[4])
    
if(ondebug): QC.barrier()

# write the result into Y
if(approach == 1):
    CCX_full(QC, A[3], A[4], Y)
if(approach == 2):
    CCX_full(QC, A[3], A[4], Y)

if(ondebug): QC.barrier()

# uncomputation
if(approach == 1):
    CCCCX(QC, A[2], X[11], X[12], X[13], A[3], "right")
    CCCX(QC, X[8], X[9], X[10], A[2], "right")
    CCX(QC, A[1], A[0], A[4])
    CCCCX(QC, X[4], X[5], X[6], X[7], A[1], "right")
    CCCCX2(QC, X[0], X[1], X[2], X[3], A[0], A[4])
if(approach == 2):
    CCX(QC, A[0], A[1], A[4])
    CCCCX(QC, A[2], X[11], X[12], X[13], A[3], "right")
    CCCX(QC, X[8], X[9], X[10], A[2], "right")
    CCCCX2(QC, X[4], X[5], X[6], X[7], A[1], A[4])
    CCCCX(QC, X[0], X[1], X[2], X[3], A[0], "right")
    
# measure
if(ondebug):
    QC.barrier()
    QC.measure(X, XC)
    QC.measure(Y, YC)
    QC.measure(A, AC)

#draw
QC.draw()

In [ ]:
from qiskit import transpile
# QC = transpile(QC, basis_gates=['h', 't', 'tdg', 'rz', 'cx'])
QC = transpile(QC, basis_gates=['u', 'cx'])
print("depth = " + str(QC.depth()))
print("gate count = " + str(QC.count_ops()))

if(ondebug):
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = backend_sim.run(transpile(QC, backend_sim), shots=65536)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(QC)
    
    dx, dy, da = {}, {}, {}
    for k, v in counts.items():
        _sa, _sy, _sx = k.split()
        sx, sy, sa = _sx[::-1], _sy[::-1], _sa[::-1]
        if("0" in sx):
            sx = "".join(["0"] * len(sx))

        dx.setdefault(sx, 0)
        dy.setdefault(sy, 0)
        da.setdefault(sa, 0)
        dx[sx] += v
        dy[sy] += v
        da[sa] += v
        if("0" not in sx and sy == "0"):
            print('sx == "111...1" && sy == "0"')
        if("0" in sx and sy == "1"):
            print('sx != "111...1" && sy == "1"')

    print("dx = " + str(dx))
    print("dy = " + str(dy))
    print("da = " + str(da))

QC.draw()

In [ ]:
# submission
QC.qasm(formatted = True)

In [ ]:
# # in debug
# depth = 61
# gate count = OrderedDict([('u', 116), ('cx', 90), ('measure', 20), ('barrier', 4)])
# dx = {'00000000000000': 65530, '11111111111111': 6}
# dy = {'0': 65530, '1': 6}
# da = {'00000': 65536}

# # submission
# depth = 57
# gate count = OrderedDict([('u', 101), ('cx', 90)])